In [19]:
import csv, sqlite3
con = sqlite3.connect("FinalDB.db")
cur = con.cursor()

In [20]:
%load_ext sql
%sql sqlite:///FinalDB.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [21]:
import pandas
df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv")
df.to_sql("CENSUS_DATA", con, if_exists='replace', index=False,method="multi")

78

In [22]:
df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv")
df.to_sql("CHICAGO_CRIME_DATA", con, if_exists='replace', index=False, method="multi")

533

In [23]:
df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv")
df.to_sql("CHICAGO_PUBLIC_SCHOOLS_DATA", con, if_exists='replace', index=False, method="multi")

566

In [24]:
%sql SELECT name FROM sqlite_master WHERE type='table'

 * sqlite:///FinalDB.db
Done.


name
CENSUS_DATA
CHICAGO_CRIME_DATA
CHICAGO_PUBLIC_SCHOOLS_DATA


In [ ]:
# Find the total number of crimes recorded in the CRIME table
%sql SELECT COUNT(*) FROM CHICAGO_CRIME_DATA

 * sqlite:///FinalDB.db
Done.


COUNT(*)
533


In [ ]:
# List community areas with per capita income less than 11000.
%sql select COMMUNITY_AREA_NAME, COMMUNITY_AREA_NUMBER from CENSUS_DATA WHERE PER_CAPITA_INCOME < 11000

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER
West Garfield Park,26.0
South Lawndale,30.0
Fuller Park,37.0
Riverdale,54.0


In [ ]:
# List all case numbers for crimes involving minors?(children are not considered minors for the purposes of crime analysis)
%sql select CASE_NUMBER from CHICAGO_CRIME_DATA WHERE DESCRIPTION LIKE "%MINOR%"

 * sqlite:///FinalDB.db
Done.


CASE_NUMBER
HL266884
HK238408


In [ ]:
# List all kidnapping crimes involving a child?
%sql SELECT * FROM CHICAGO_CRIME_DATA WHERE PRIMARY_TYPE = 'KIDNAPPING' AND DESCRIPTION LIKE '%CHILD%'

 * sqlite:///FinalDB.db
Done.


ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,0,0,1533,15,29.0,25.0,20,1143050.0,1897546.0,2007,41.87490841,-87.75024931,"(41.874908413, -87.750249307)"


In [ ]:
# List the kind of crimes that were recorded at schools. (No repetitions)
%sql SELECT DISTINCT(PRIMARY_TYPE) FROM CHICAGO_CRIME_DATA WHERE LOCATION_DESCRIPTION LIKE '%SCHOOL%'

 * sqlite:///FinalDB.db
Done.


PRIMARY_TYPE
BATTERY
CRIMINAL DAMAGE
NARCOTICS
ASSAULT
CRIMINAL TRESPASS
PUBLIC PEACE VIOLATION


In [ ]:
# List the type of schools along with the average safety score for each type.
%%sql select "Elementary, Middle, or High School", AVG(SAFETY_SCORE)
from CHICAGO_PUBLIC_SCHOOLS_DATA GROUP BY "Elementary, Middle, or High School"

 * sqlite:///FinalDB.db
Done.


"Elementary, Middle, or High School",AVG(SAFETY_SCORE)
ES,49.52038369304557
HS,49.62352941176471
MS,48.0


In [ ]:
# List 5 community areas with highest % of households below poverty line
%%sql select COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY
from CENSUS_DATA ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC LIMIT 5

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


In [ ]:
# Which community area is most crime prone? Display the coumminty area number only
%%sql select COMMUNITY_AREA_NUMBER from CHICAGO_CRIME_DATA GROUP BY COMMUNITY_AREA_NUMBER
ORDER BY COUNT(COMMUNITY_AREA_NUMBER) DESC LIMIT 1

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER
25.0


In [ ]:
# Use a sub-query to find the name of the community area with highest hardship index
%sql SELECT COMMUNITY_AREA_NAME FROM CENSUS_DATA WHERE HARDSHIP_INDEX = (SELECT MAX(HARDSHIP_INDEX) FROM CENSUS_DATA)

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Riverdale


In [ ]:
# Use a sub-query to determine the Community Area Name with most number of crimes?
%%sql SELECT COMMUNITY_AREA_NAME FROM CENSUS_DATA WHERE COMMUNITY_AREA_NUMBER = (SELECT COMMUNITY_AREA_NUMBER
FROM CHICAGO_CRIME_DATA GROUP BY COMMUNITY_AREA_NUMBER ORDER BY COUNT(*) DESC LIMIT 1)

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Austin


In [30]:
# Write and execute a SQL query to list the school names, community names and average attendance for communities with a hardship index of 98.
%%sql SELECT CPS.Name_of_School AS School_Name, CD.COMMUNITY_AREA_NUMBER AS Community_Name, AVG(CPS.Average_Student_Attendance) AS Average_Attendance
FROM CHICAGO_PUBLIC_SCHOOLS_DATA CPS
LEFT JOIN CENSUS_DATA CD ON CPS.Community_Area_Number = CD.COMMUNITY_AREA_NUMBER
WHERE CD.HARDSHIP_INDEX = 98
GROUP BY CPS.Name_of_School, CD.COMMUNITY_AREA_NUMBER;

 * sqlite:///FinalDB.db
Done.


School_Name,Community_Name,Average_Attendance
George Washington Carver Military Academy High School,54.0,91.6
George Washington Carver Primary School,54.0,90.9
Ira F Aldridge Elementary School,54.0,92.9
William E B Dubois Elementary School,54.0,93.3


In [36]:
# Write and execute a SQL query to list all crimes that took place at a school. Include case number, crime type and community name
%%sql SELECT CCD.Case_Number, CCD.Primary_Type AS Crime_Type, CD.COMMUNITY_AREA_NAME AS Community_Name
FROM CHICAGO_CRIME_DATA CCD
JOIN CENSUS_DATA CD ON CCD.Community_Area_Number = CD.COMMUNITY_AREA_NUMBER
WHERE CCD.Location_Description LIKE '%SCHOOL%'

 * sqlite:///FinalDB.db
Done.


CASE_NUMBER,Crime_Type,Community_Name
HL353697,BATTERY,South Shore
HL725506,BATTERY,Lincoln Square
HP716225,BATTERY,Douglas
HH639427,BATTERY,Austin
JA460432,BATTERY,Ashburn
HS200939,CRIMINAL DAMAGE,Austin
HK577020,NARCOTICS,Rogers Park
HS305355,NARCOTICS,Brighton Park
HT315369,ASSAULT,East Garfield Park
HR585012,CRIMINAL TRESPASS,Ashburn


In [40]:
%%sql CREATE VIEW School_Ratings_View AS
SELECT
    NAME_OF_SCHOOL AS School_Name,
    Safety_Icon AS Safety_Rating,
    Family_Involvement_Icon AS Family_Rating,
    Environment_Icon AS Environment_Rating,
    Instruction_Icon AS Instruction_Rating,
    Leaders_Icon AS Leaders_Rating,
    Teachers_Icon AS Teachers_Rating
FROM
    CHICAGO_PUBLIC_SCHOOLS_DATA;

 * sqlite:///FinalDB.db
Done.


[]

In [ ]:
%%sql SELECT * FROM School_Ratings_View;

In [ ]:
%sql SELECT School_Name, Leaders_Rating FROM School_Ratings_View;